## Experiment concerning the Simpson's Paradox in hyppthetical university admission data
#### The following jupyter notebook accompanies the seminar paper "Methods to correct an imbalanced dataset" by Johannes Winkler. It generates an imbalanced dataset of hypothetical students applying to a university that exhibits the so-called Simpson's Paradox. Different resmampling methods are applied to the data and the admission probabilities for female and male students are calculated again. The comparison of the (conditional) admission probabilites before and after applying the resampling methods sheds some light on wether these increase the Simpson's Paradox.

In [12]:
# load the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import TomekLinks

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

from imblearn.combine import SMOTETomek
from imblearn.combine import SMOTEENN

from sklearn.datasets import make_classification

In [13]:
# non-linear function to calculate cut-off values given 4 arbitrary features
def new_f(vector):
    result =[]
    for i in range(vector.shape[0]):
        result.append(np.sin(vector[i,0]) * np.power(vector[i, 1],3) * np.power(vector[i, 2], 2) * max(vector[i,3],-0.5))

    return result 

In [14]:
# generate data 
# number of observations = no_samples
# number of classes = 2
# number of features each data points consists of = 4
# number of cluster per class = 2
# size of each cluster: class= 0 : 80%, class = 1: 20%
# flip_y = 0% defines error rate in class assignment
no_samples = 100000
X, y = make_classification(n_samples=no_samples, n_features=4, n_redundant=0, n_classes=2, n_clusters_per_class=2, weights=[0.8, 0.2 ], flip_y=0.0, random_state=42)
z = new_f(X)

In [15]:
# determine cut-off values
zz = np.array(z) # keep vector z
zz = np.sort(zz)
cut_off_no_eng_man = zz[30000] # male student is accepted if value is below cut-off, i.e. 30% when applying to social studies
cut_off_no_eng_woman = zz[32000] # female student is accepted if value is below cut-off , i.e. 32% when applying to social studies
cut_off_eng_male = zz[40000] # male student is accepted in engineering department with 40% likelihood
cut_off_eng_female = zz[42000] # female student is accepted in engineering department with 42% likelihood

In [16]:
# print cut-off values
cut_off_no_eng_man, cut_off_no_eng_woman

(-0.00791399421882229, -0.005051396762245379)

In [17]:
# randomly assign students to departments
# a student is accepted if z-value is below the respective cut-off value

accepted = []
eng_department = []
for i in range(no_samples):
    func_value = z[i]
    if y[i]: # female student
        # wants to study math:
        math = np.random.choice([0, 1], size=(1,), p=[0.8, 0.2]) # to which department to apply to 
        # engineering department
        if math:
            eng_department.append(1)
            if func_value < cut_off_eng_female: 
                accepted.append(1)
                # accepted
            else:
                accepted.append(0)
                # not accepted
        else: # social studies
            eng_department.append(0)
            if func_value < cut_off_no_eng_woman: 
                accepted.append(1)
                # accepted
            else:
                accepted.append(0)
                # not accepted
    else: # male student
        math = np.random.choice([0, 1], size=(1,), p=[0.2, 0.8]) # to which department to apply to 
        if math: # engineering department
            eng_department.append(1)
            if func_value < cut_off_eng_male: 
                # accepted
                accepted.append(1)
            else:
                # not accepted
                accepted.append(0)
        else:
            eng_department.append(0)
            if func_value < cut_off_no_eng_man: 
                accepted.append(1)
                # accepted
            else:
                accepted.append(0)
                # not accepted


In [18]:
# calculate unconditional probability of acceptance for male and female students
df = pd.DataFrame({'Female': y, 'Accepted': np.array(accepted),'Engineering': np.array(eng_department)})
male = df[df['Female']==0]
female = df[df['Female']==1]
no_male_accepted = male[male['Accepted']==1].shape[0]
no_female_accepted = female[female['Accepted']==1].shape[0]
accept_prob_man = 100*no_male_accepted /male.shape[0]
accept_prob_woman = 100*no_female_accepted /female.shape[0]
print(f'chance of admission male: {round(accept_prob_man,1)}')
print(f'chance of admission male female: {round(accept_prob_woman,1)}')

chance of admission male: 37.8
chance of admission male female: 35.3


In [19]:
engineering = df[df['Engineering']==1]
male_engineering = engineering[engineering['Female']==0]
female_engineering = engineering[engineering['Female']==1]
no_male_engineering_accepted = male_engineering[male_engineering['Accepted']==1].shape[0]
no_female_engineering_accepted = female_engineering[female_engineering['Accepted']==1].shape[0]
prob_male_eng = 100*no_male_engineering_accepted /male_engineering.shape[0]
prob_female_eng = 100*no_female_engineering_accepted /female_engineering.shape[0]
print(f'probability of admission for men in engineering dep.: {round(prob_male_eng,1)}')
print(f'probability of admission for women in engineering dep.: {round(prob_female_eng,1)}')
## social studies
no_engineering = df[df['Engineering']==0]
male_no_engineering = no_engineering[no_engineering['Female']==0]
female_no_engineering = no_engineering[no_engineering['Female']==1]
no_male_no_engineering_accepted = male_no_engineering[male_no_engineering['Accepted']==1].shape[0]
no_female_no_engineering_accepted = female_no_engineering[female_no_engineering['Accepted']==1].shape[0]

print(f'probability of admission for men in social studies dep.: {round(100*no_male_no_engineering_accepted /male_no_engineering.shape[0],1)}')
print(f'probability of admission for men in social studies dep.: {round(100*no_female_no_engineering_accepted /female_no_engineering.shape[0],1)}')

probability of admission for men in engineering dep.: 39.8
probability of admission for women in engineering dep.: 42.2
probability of admission for men in social studies dep.: 29.5
probability of admission for men in social studies dep.: 33.6


In [20]:
# create dataframe of features and flags for engineering and gender
new_X = pd.DataFrame({'Feature_1': X[:,0],'Feature_2': X[:,1],'Feature_3': X[:,2],'Feature_4': X[:,3],'Engineering': df['Engineering'],'Female': df['Female']})
new_X.head()

,Feature_1,Feature_2,Feature_3,Feature_4,Engineering,Female
0,-1.488129,1.528193,0.620234,-0.823358,1,0
1,1.553788,-0.033152,0.422981,0.505014,1,1
2,1.707101,1.916607,-0.076054,-1.483083,1,0
3,1.196007,-1.074387,0.087178,-0.509525,1,0
4,1.060068,-1.412220,0.839212,-1.439767,0,0


In [21]:
# set seed for random number generator to make results replicable
RandomState = 42

# instantiate resampling methods
sample_RUS = RandomUnderSampler(random_state=RandomState)
sample_ROS = RandomOverSampler(random_state=RandomState)
sample_TL = TomekLinks()
sample_ADASYN =  ADASYN(random_state=RandomState)
sample_SMOTE = SMOTE(random_state=RandomState)
sample_ENN = EditedNearestNeighbours()
sample_ST = SMOTETomek(random_state=RandomState)
sample_STEEN = SMOTEENN(random_state=RandomState)

In [23]:
# students randomly aplly to departments are admitted or denied


sample = [sample_ROS, sample_RUS, sample_TL, sample_ADASYN, sample_SMOTE, sample_ENN, sample_ST, sample_STEEN]

for sam in sample:
    sample_X, sample_y = sam.fit_resample(new_X[['Feature_1','Feature_2','Feature_3','Feature_4','Engineering']], new_X['Female'])
    sample_X.reset_index(drop=True, inplace=True)
    sample_y.reset_index(drop=True, inplace=True)
    sample_z = new_f(np.array(sample_X.iloc[:,0:4]))

    new_accepted = []

    for i in range(sample_X.shape[0]):
        func_value = sample_z[i] 
        if sample_X.Engineering[i]: # in engineering department
            if sample_y[i]: # if woman
                if func_value < cut_off_eng_female: 
                    new_accepted.append(1)
                    #print('accepted')
                else:
                    new_accepted.append(0)
                    #print('NOT accepted')
            else: # if man
                if func_value < cut_off_eng_male: 
                    new_accepted.append(1)
                    #print('accepted')
                else:
                    new_accepted.append(0)
                    #print('NOT accepted')
        else:
            if sample_y[i]: # if woman
                if func_value < cut_off_no_eng_woman: 
                    new_accepted.append(1)
                #print('accepted')
                else:
                    new_accepted.append(0)
                    #print('NOT accepted')
            else:
                if func_value < cut_off_no_eng_man: 
                    new_accepted.append(1)
                #print('accepted')
                else:
                    new_accepted.append(0)
                    #print('NOT accepted')

    new_df = pd.DataFrame({'Female': sample_y, 'Accepted': np.array(new_accepted), 'Engineering': np.array(sample_X.Engineering)})

    #Output
    print('**********************')
    print(f'resampling method : {sam}')
    print(f'number of students: {new_df.shape[0]}')
    male = new_df[new_df['Female']==0]
    female = new_df[new_df['Female']==1]
    no_male_accepted = male[male['Accepted']==1].shape[0]
    no_female_accepted = female[female['Accepted']==1].shape[0]
    sam_accept_prob_man = 100*no_male_accepted /male.shape[0]
    sam_accept_prob_woman = 100*no_female_accepted /female.shape[0]
    print(f'prob. of admission men: {round(sam_accept_prob_man,1)}, difference: {round(sam_accept_prob_man - accept_prob_man,1)}')
    print(f'prob of admission women: {round(sam_accept_prob_woman,1)}, difference: {round(sam_accept_prob_woman - accept_prob_woman,1)}')
    eng = new_df[new_df['Engineering']==1]
    eng_male = eng[eng['Female']==0]
    eng_female = eng[eng['Female']==1]
    eng_no_male_accepted = eng_male[eng_male['Accepted']==1].shape[0]
    eng_no_female_accepted = eng_female[eng_female['Accepted']==1].shape[0]
    sam_eng_accept_prob_man = 100*eng_no_male_accepted /eng_male.shape[0]
    sam_eng_accept_prob_woman = 100*eng_no_female_accepted /eng_female.shape[0]
    print(f'prob admission men in engineering dep.: {round(sam_eng_accept_prob_man,1)}, difference:{round(sam_eng_accept_prob_man - prob_male_eng,1)} ')
    print(f'prob admission women in engineering dep.: {round(sam_eng_accept_prob_woman,1)}, difference:{round(sam_eng_accept_prob_woman - prob_female_eng,1)} ')
    
    print('**********************')
    print('')


**********************
resampling method : RandomOverSampler(random_state=42)
number of students: 160000
prob. of admission men: 37.8, difference: 0.0
prob of admission women: 35.3, difference: 0.0
prob admission men in engineering dep.: 39.8, difference:0.0 
prob admission women in engineering dep.: 42.1, difference:-0.1 
**********************

**********************
resampling method : RandomUnderSampler(random_state=42)
number of students: 40000
prob. of admission men: 37.7, difference: -0.0
prob of admission women: 35.3, difference: 0.0
prob admission men in engineering dep.: 39.7, difference:-0.1 
prob admission women in engineering dep.: 42.2, difference:0.0 
**********************

**********************
resampling method : TomekLinks()
number of students: 97380
prob. of admission men: 38.0, difference: 0.2
prob of admission women: 35.3, difference: 0.0
prob admission men in engineering dep.: 39.8, difference:0.0 
prob admission women in engineering dep.: 42.2, difference:0.0 
